In [1]:
from window_generator import WindowGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
from read_data import *

MAX_EPOCHS = 20

In [2]:
def compile_and_fit(model, window, patience=2, verbose=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')

    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.MeanAbsoluteError()])

# At the end of each epoch, the model will iterate over the validation dataset and compute the validation loss and validation metrics.
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping],
                        verbose=verbose)
    return history

In [3]:
multi_step_window = WindowGenerator(input_width=6, label_width=1, shift=1)

for example_inputs, example_labels in multi_step_window.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

print(multi_step_window.train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Inputs shape (batch, time, features): (32, 6, 6)
Labels shape (batch, time, features): (32, 1, 22)
<MapDataset shapes: ((None, 6, 6), (None, 1, 22)), types: (tf.float32, tf.float32)>


In [4]:
linear = tf.keras.Sequential(
    [tf.keras.layers.Dense(22, activation="softmax")])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', linear(multi_step_window.example[0]).shape)

Input shape: (32, 6, 6)
Output shape: (32, 6, 22)


In [5]:
linear_multi_step = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=32, activation='relu'),
#     tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=22, activation='softmax'),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', linear_multi_step(multi_step_window.example[0]).shape)

Input shape: (32, 6, 6)
Output shape: (32, 1, 22)


In [6]:
# Train the model
history = compile_and_fit(linear_multi_step, multi_step_window, verbose=0)
print(
    "loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
        history.history["loss"][-1],
        history.history["mean_absolute_error"][-1],
        history.history["val_loss"][-1],
        history.history["val_mean_absolute_error"][-1],
    )
)

loss: 0.0169, mean_absolute_error: 0.0377, val_loss: 0.0210, val_mean_absolute_error: 0.0432


In [7]:
# Evaluate the model
print("Evaluate")
result = linear_multi_step.evaluate(multi_step_window.test)
dict(zip(linear_multi_step.metrics_names, result))

Evaluate
12/12 [==============================] - 0s 4ms/step - loss: 0.0263 - mean_absolute_error: 0.0495


{'loss': 0.02626441791653633, 'mean_absolute_error': 0.049481768161058426}

In [17]:
# Get last window 
last_window = None
last_pitch = None
all_predicted_pitches = []

for i in range(100):
    if last_window is None:
        for inputs, labels in multi_step_window.test_no_shuffle.as_numpy_iterator():
            pass

#     print('window:', type(inputs),inputs ,'\n====\nlabels', labels)
#     print('window shape:', inputs.shape, ' label shape:', labels.shape)

    # Model is expecting a shape of (batch, timestep, features) but for one sample this is (6,6)
    # Add dimension to input
    probabilities = linear_multi_step.predict( np.array([inputs]))[0]
    probabilities = probabilities[0]
    predicted_pitch = get_pitch_from_probability(probabilities, key=multi_step_window.pitch_conversion_key)
    all_predicted_pitches.append(predicted_pitch)
    features = get_pitch_features(predicted_pitch)

    if last_pitch is None:
        last_pitch = get_voice(0)[-1]

    # - find new duration based on previous and current pitch
    if last_pitch == predicted_pitch:
        duration = multi_step_window.df.iloc[-1]['dur'] + 1 
    else:
        duration = 1
        
    last_pitch = predicted_pitch

    # create new dataframe
    new_df = pd.DataFrame(data={'dur': duration}, dtype=float, index=[0])
    new_df['log_pitch'], new_df['chroma_x'], new_df['chroma_y'], new_df['c5_x'], new_df['c5_y'] = features
    # - Normalise features
    new_df = (new_df - multi_step_window.train_mean) / multi_step_window.train_std
    # - Make new window based on previous timesteps with a slide of 1 and the normalised features
    last_window = np.append(inputs[1:], new_df.values.tolist(), axis=0)

print(all_predicted_pitches)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
